Сравнение разных детекторов досок по validation loss

In [1]:
cd ..

/home/bfilippov/kaggle/whiteboard


In [2]:
import keras
import pandas as pd
import ast
import numpy as np 
import cv2 


train = pd.read_csv('source/train.csv')
test = pd.read_csv('source/test.csv')
train['labels'] = train['labels'].map(ast.literal_eval)
test['labels'] = test['labels'].map(ast.literal_eval)

valid = train.sample(frac=0.2, random_state=1313)
train = train.drop(valid.index)

Using TensorFlow backend.


In [3]:
nb_cv_features = 5

def cv_feature_maps(image):
    alpha = image[:, :, 3].copy() # special mask used to ignore image-augmentation artifacts
    gray = (cv2.cvtColor(image[:, :, :3].astype(np.float32), cv2.COLOR_RGB2GRAY) * 255).astype(np.uint8)
    edges = cv2.Canny(gray, 100, 250)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 25, minLineLength=50, maxLineGap=50)

    corners = cv2.cornerHarris(edges,2,3,0.01)
    corners = cv2.dilate(corners,None)
    
    hough = np.zeros(gray.shape, np.float)

    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(hough, (x1, y1), (x2, y2), 1, 2)
    
    def norm(img):
        return cv2.normalize(img, img, 0, 1, norm_type=cv2.NORM_MINMAX, 
                             dtype=cv2.CV_32F)
    feature_maps = [norm(img) for img in [gray, edges, corners, hough, alpha]]
    return np.stack(feature_maps, axis=2)

In [4]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf

from keras import backend as K

sess = tf.Session()
K.set_session(sess)

In [5]:
class PatchLabelIterator:
    line_thickness = 2
    def __init__(self, whiteboard_images, model):
        self.whiteboard_images = whiteboard_images
        self.out_size = model.out_size[::-1]
        self.patch_size = model.patch_size
        
    def __point(self, batch, k):
        x_val = int(batch[k] // self.patch_size)
        y_val = int(batch[k + 4] // self.patch_size)            
        return x_val, y_val
    
    def __next__(self):
        batch_x, batch_y = next(self.whiteboard_images)
        patch_x = np.zeros(batch_x.shape[:-1] + (nb_cv_features,))
        patch_y = np.zeros((batch_y.shape[0],) + self.out_size + (1,))
        for i, batch in enumerate(batch_y):
            patch_x[i] = cv_feature_maps(batch_x[i])
            if batch[0]:
                for a, b in [(2, 3), (3, 4), (4, 5), (5, 2)]:
                    cv2.line(patch_y[i], self.__point(batch_y[i], a), 
                             self.__point(batch_y[i], b), 1, self.line_thickness)
        return patch_x, patch_y

In [6]:
from models.image_generator import whiteboard_images

def validate(model):
    img_size = (model.height, model.width)
    img_dir = 'source'
    batch_size = 32
    val_samples = 2048

    def patch(gen):
        return PatchLabelIterator(gen, model)

    valid_imgs = patch(whiteboard_images(valid, img_dir, img_size, batch_size=batch_size, seed=1313))

    return model.model.evaluate_generator(valid_imgs, val_samples)

In [7]:
from models.model_zoo import wide_whiteboard_detector, baseline_whiteboard_detector

validate(baseline_whiteboard_detector())

0.27243718504905701

In [8]:
validate(wide_whiteboard_detector())

0.27417239546775818

In [7]:
from models.model_zoo import residual_detector_4_blocks

validate(residual_detector_4_blocks())

0.32326844334602356

In [7]:
from models.model_zoo import residual_detector_6_blocks

validate(residual_detector_6_blocks())

0.20618212223052979